<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/few_shot_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deeppavlov

     |████████████████████████████████| 993kB 7.5MB/s 
     |████████████████████████████████| 8.2MB 9.2MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 860kB 51.5MB/s 
     |████████████████████████████████| 10.4MB 41.5MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 2.1MB 47.5MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 655kB 46.7MB/s 
     |████████████████████████████████| 6.7MB 25.6MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 512kB 41.2MB/s 
     |████████████████████████████████| 1.5MB 40.1MB/s 
     |████████████████████████████████| 20.1MB 66.0MB/s 
     |████████████████████████████████| 7.3MB 48.3MB/s 
     |████████████████████████████████| 3.8MB 33.9MB/s 
    

In [2]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 32kB/s 
     |████████████████████████████████| 512kB 37.2MB/s 
     |████████████████████████████████| 3.8MB 38.1MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=73c7e68ec970dcc1c4b074a0f0c917e8db442f782a9dc10bb2ae300021c51868
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [3]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [4]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-xxe_li1z
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-xxe_li1z
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=12bcab8995a9e8e008c427917a99e2c474e9f6d4c8e091675e505d81ee99f7a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ax2lq94p/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [5]:
# # !rm few_test.txt
# !rm checkpoint  model.data-00000-of-00001  model.meta   tag.dict  train.txt logs model.index sample_data test.txt valid.txt
# !ls

In [6]:
!rm train.txt train.txt.1

rm: cannot remove 'train.txt': No such file or directory
rm: cannot remove 'train.txt.1': No such file or directory


In [7]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/train.txt

--2021-04-12 15:52:33--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45807 (45K) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]  44.73K  --.-KB/s    in 0.007s  

2021-04-12 15:52:34 (6.23 MB/s) - ‘train.txt’ saved [45807/45807]



In [8]:
!mv train.txt tmp_train.txt
!ls

sample_data  tmp_train.txt


In [9]:
print(open('tmp_train.txt').readlines()[0].split())
open('tmp_train.txt').readlines()[10].split()

['клинические', '0']


['карипразин', 'B-MED']

In [10]:
# handler = open('tmp_train.txt')
# wr = open('train.txt', 'w')
# # print(len(handler.readlines()))
# for i in handler.readlines():
#   i = i.replace('\t', ' ')
#   i = i.replace('O', '0')
#   wr.write(i)
# wr.close()
# handler.close

In [11]:
'  '.strip()

''

In [20]:
import re

In [12]:
handler = open('tmp_train.txt')
wr = open('train.txt', 'w')
ll = ['B-MED', 'I-MED', 'B-EFFECT', 'I-EFFECT', 'B-SICK', 'I-SICK', '0', 0]
# print(len(handler.readlines()))
for i in handler.readlines():
  i = i.replace('\t', ' ')
  if i == '\n':
    # print('yep')
    wr.write('\n')
  if i.split() and len(i.split()) > 1:
    if i.split()[0].strip() and i.split()[1] in ll:
      wr.write(f'{i.split()[0]} {i.split()[1]}\n')
wr.close()
handler.close

<function TextIOWrapper.close>

In [13]:
print(len(open('train.txt').readlines()))

3274


In [14]:
!rm tmp_train.txt
!ls

sample_data  train.txt


In [15]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/test.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/valid.txt

--2021-04-12 15:52:34--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5188 (5.1K) [text/plain]
Saving to: ‘test.txt’

test.txt            100%[===================>]   5.07K  --.-KB/s    in 0s      

2021-04-12 15:52:35 (60.5 MB/s) - ‘test.txt’ saved [5188/5188]

--2021-04-12 15:52:35--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/few_shot_bert/valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [16]:
# !cp train.txt test.txt
# !cp train.txt valid.txt
!rm -r logs  sample_data
!ls

rm: cannot remove 'logs': No such file or directory
test.txt  train.txt  valid.txt


In [17]:

import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

In [18]:
# ner_config
open('train.txt').readlines()[:3]

['клинические 0\n', 'исследования 0\n', 'в 0\n']

In [19]:
ner_model = train_model(ner_config, download=True)

2021-04-12 15:52:37.715 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [02:24<00:00, 4.60MB/s]
2021-04-12 15:55:03.179 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models


Exception: ignored

In [ ]:
ner_model(["какое-то не лекарство", "пью леонезит 1 месяц", "варкрафт 2 интересная игра"])

In [ ]:
nanika = ner_model(['Паксил норм. Принимал около года, щас перерыв. \
  Синдром отмены не то чтобы совсем отсутствует, но мне особых непрятностей не доставил. \
  Еще тироксин в качестве антидепрессанта хорош, хотя формально показания у него другие.'])

In [ ]:
list(zip(nanika[0][0], nanika[1][0]))

In [ ]:
from collections import Counter

In [ ]:
Counter('ttt hhh kkk')

In [ ]:
list(Counter('ttt hhh kkk').elements())